# Music Generation Using Deep Learning

In [8]:
from music21 import *
import glob
import numpy as np

## Loading and Preprocessing Data

In [9]:
# loading the MIDI Files

    
notes = []

for file in glob.glob("piano_midi/*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else:
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))
            
            

    

In [10]:
len(notes)

57177

In [11]:
notes

['4.9',
 'E2',
 '4.9',
 '4.9',
 '4.9',
 '4.9',
 '4.9',
 '4.9',
 '4.9',
 '11.4',
 '4.9',
 '11.4',
 '4.9',
 '4.9',
 '4.9',
 '4.9',
 '4.9',
 '0.4',
 'E2',
 '4.9',
 '0.4',
 '4.9',
 '4.9',
 '4.9',
 '4.9',
 '4.9',
 '9.2',
 '4.9',
 '9.2',
 '9.2',
 '4.9',
 '4.9',
 '4.9',
 '4.9',
 '4.9',
 '4.9',
 'E2',
 '4.9',
 '4.9',
 '4.9',
 '4.9',
 '4.9',
 'E5',
 'F5',
 'G#5',
 'A5',
 '4.9',
 '4.9',
 '5.11',
 '4.9',
 '5.11',
 '4.9',
 '4.9',
 '4.9',
 'E5',
 'F5',
 'G#5',
 'A5',
 '4.9',
 '4.9',
 '9.0',
 'E2',
 '4.9',
 '9.0',
 '4.9',
 '4.9',
 '4.9',
 'E5',
 'F5',
 'G#5',
 'A5',
 '4.9',
 '4.9',
 '11.2',
 '4.9',
 '11.2',
 '11.2',
 '4.9',
 '4.9',
 '4.9',
 'E5',
 'F5',
 'G#5',
 'A5',
 '4.9',
 '4.9',
 '3.7.11',
 'E-2',
 '3.7.11',
 'B2',
 'G2',
 '1.5.9',
 'F#2',
 '1.5.9',
 '3.7.11',
 'E-2',
 '3.7.11',
 'G2',
 'B2',
 'E-3',
 '1.5.9',
 'G#2',
 '1.5.9',
 '1.5.9',
 'F3',
 'F2',
 'F2',
 'F2',
 'F2',
 'F2',
 '4.9',
 'E5',
 '4.9',
 'C5',
 '4.9',
 'A5',
 '4.9',
 '5.9',
 'F5',
 '5.9',
 'C5',
 '5.9',
 'A5',
 '5.9',
 '4.9',
 'E

In [12]:
#defining sequence length
sequence_len = 100


In [13]:
#get all the pitches

pitches = sorted(set(item for item in notes))

In [14]:
pitches

['0',
 '0.1',
 '0.1.5',
 '0.1.6',
 '0.2',
 '0.2.3.7',
 '0.2.4.7',
 '0.2.5',
 '0.2.6',
 '0.2.7',
 '0.3',
 '0.3.5',
 '0.3.5.8',
 '0.3.6',
 '0.3.6.8',
 '0.3.6.9',
 '0.3.7',
 '0.4',
 '0.4.5',
 '0.4.6',
 '0.4.7',
 '0.5',
 '0.5.6',
 '0.6',
 '1',
 '1.2',
 '1.2.4.6.8.10',
 '1.2.6',
 '1.2.6.8',
 '1.3',
 '1.3.5',
 '1.3.5.8',
 '1.3.6',
 '1.3.7',
 '1.3.8',
 '1.4',
 '1.4.6',
 '1.4.6.9',
 '1.4.7',
 '1.4.7.10',
 '1.4.7.9',
 '1.4.8',
 '1.5',
 '1.5.8',
 '1.5.9',
 '1.6',
 '1.7',
 '10',
 '10.0',
 '10.0.2.5',
 '10.0.3',
 '10.0.4',
 '10.0.5',
 '10.1',
 '10.1.3',
 '10.1.3.5.6',
 '10.1.3.6',
 '10.1.4',
 '10.1.4.6',
 '10.1.5',
 '10.11',
 '10.11.3',
 '10.11.3.5',
 '10.2',
 '10.2.3',
 '10.2.4',
 '10.2.5',
 '10.3',
 '11',
 '11.0',
 '11.0.4',
 '11.0.4.6',
 '11.0.4.7',
 '11.0.5',
 '11.1',
 '11.1.4',
 '11.1.4.5',
 '11.1.5',
 '11.1.6',
 '11.2',
 '11.2.4',
 '11.2.4.6',
 '11.2.4.7',
 '11.2.5',
 '11.2.5.7',
 '11.2.6',
 '11.3',
 '11.3.5',
 '11.3.6',
 '11.4',
 '11.4.5',
 '2',
 '2.3',
 '2.3.7',
 '2.3.7.10',
 '2.3.7.9',
 '

In [15]:
#create a integer for every note

note_to_int = dict()

for note, id in enumerate(pitches):
    note_to_int[id] = note
    
    

In [10]:
note_to_int

{'0': 0,
 '0.1': 1,
 '0.1.5': 2,
 '0.1.6': 3,
 '0.2': 4,
 '0.2.3.7': 5,
 '0.2.4.7': 6,
 '0.2.5': 7,
 '0.2.6': 8,
 '0.2.7': 9,
 '0.3': 10,
 '0.3.5': 11,
 '0.3.5.8': 12,
 '0.3.6': 13,
 '0.3.6.8': 14,
 '0.3.6.9': 15,
 '0.3.7': 16,
 '0.4': 17,
 '0.4.5': 18,
 '0.4.6': 19,
 '0.4.7': 20,
 '0.5': 21,
 '0.5.6': 22,
 '0.6': 23,
 '1': 24,
 '1.2': 25,
 '1.2.4.6.8.10': 26,
 '1.2.6': 27,
 '1.2.6.8': 28,
 '1.3': 29,
 '1.3.5': 30,
 '1.3.5.8': 31,
 '1.3.6': 32,
 '1.3.7': 33,
 '1.3.8': 34,
 '1.4': 35,
 '1.4.6': 36,
 '1.4.6.9': 37,
 '1.4.7': 38,
 '1.4.7.10': 39,
 '1.4.7.9': 40,
 '1.4.8': 41,
 '1.5': 42,
 '1.5.8': 43,
 '1.5.9': 44,
 '1.6': 45,
 '1.7': 46,
 '10': 47,
 '10.0': 48,
 '10.0.2.5': 49,
 '10.0.3': 50,
 '10.0.4': 51,
 '10.0.5': 52,
 '10.1': 53,
 '10.1.3': 54,
 '10.1.3.5.6': 55,
 '10.1.3.6': 56,
 '10.1.4': 57,
 '10.1.4.6': 58,
 '10.1.5': 59,
 '10.11': 60,
 '10.11.3': 61,
 '10.11.3.5': 62,
 '10.2': 63,
 '10.2.3': 64,
 '10.2.4': 65,
 '10.2.5': 66,
 '10.3': 67,
 '11': 68,
 '11.0': 69,
 '11.0.4': 70,
 

In [16]:
#creating input and output data

X = []
y = []

#getting the sequence and mapping it with the interger from note_to_int
for i in range(len(notes)-sequence_len):
    input_seq = notes[i:i+sequence_len]
    
    output_seq = notes[i+sequence_len]
    
    #mapping with intgers
    X.append([note_to_int[char] for char in input_seq])
    y.append(note_to_int[output_seq])

In [17]:
print('Length of X')
len(X)

Length of X


57077

In [18]:
print('Length of y')
len(y)

Length of y


57077

In [19]:
len(X[0])

100

In [20]:
n_size = len(X)

In [16]:
y

[44,
 347,
 44,
 44,
 342,
 341,
 341,
 341,
 341,
 341,
 168,
 332,
 168,
 312,
 168,
 286,
 168,
 183,
 344,
 183,
 312,
 183,
 286,
 183,
 168,
 332,
 168,
 312,
 168,
 286,
 168,
 344,
 183,
 312,
 183,
 332,
 183,
 319,
 183,
 332,
 168,
 326,
 168,
 299,
 168,
 168,
 286,
 183,
 183,
 183,
 183,
 168,
 168,
 168,
 168,
 183,
 183,
 183,
 183,
 298,
 168,
 285,
 168,
 332,
 168,
 168,
 326,
 183,
 183,
 183,
 183,
 168,
 168,
 168,
 168,
 183,
 183,
 183,
 183,
 332,
 143,
 326,
 313,
 332,
 170,
 326,
 299,
 332,
 187,
 326,
 313,
 286,
 170,
 285,
 143,
 312,
 332,
 344,
 332,
 170,
 312,
 285,
 312,
 285,
 187,
 312,
 332,
 338,
 332,
 170,
 312,
 285,
 312,
 285,
 143,
 312,
 332,
 344,
 332,
 170,
 312,
 285,
 312,
 285,
 187,
 312,
 332,
 338,
 332,
 170,
 312,
 285,
 312,
 285,
 283,
 312,
 332,
 344,
 283,
 332,
 283,
 312,
 298,
 312,
 285,
 283,
 343,
 331,
 343,
 283,
 331,
 283,
 311,
 297,
 311,
 332,
 168,
 299,
 168,
 286,
 168,
 356,
 344,
 168,
 168,
 332,
 171,
 

In [21]:
#Shaping the data for LSTM
from keras.utils import np_utils

n_vocab = len(set(notes)) #finding number of unique notes

X = np.reshape(X, (n_size, sequence_len, 1))

y = np_utils.to_categorical(y) #one hot encoding of the output sequence

Using TensorFlow backend.


In [22]:
X.shape

(57077, 100, 1)

In [23]:
y.shape

(57077, 358)

In [24]:
X.shape[1]

100

In [25]:
n_vocab

358

In [26]:
#Normalizing Inputs

X = X / float(n_vocab)
X

array([[[0.46927374],
        [0.91899441],
        [0.46927374],
        ...,
        [0.98603352],
        [0.82681564],
        [0.90502793]],

       [[0.91899441],
        [0.46927374],
        [0.46927374],
        ...,
        [0.82681564],
        [0.90502793],
        [0.12290503]],

       [[0.46927374],
        [0.46927374],
        [0.46927374],
        ...,
        [0.90502793],
        [0.12290503],
        [0.96927374]],

       ...,

       [[0.32402235],
        [0.32122905],
        [0.44413408],
        ...,
        [0.17597765],
        [0.98882682],
        [0.04748603]],

       [[0.32122905],
        [0.44413408],
        [0.76536313],
        ...,
        [0.98882682],
        [0.04748603],
        [0.88547486]],

       [[0.44413408],
        [0.76536313],
        [0.92178771],
        ...,
        [0.04748603],
        [0.88547486],
        [0.32122905]]])

## Model Building and Training

In [27]:
#Defining the model

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout,LSTM,Activation,BatchNormalization
from keras.callbacks import ModelCheckpoint

In [28]:
model = Sequential()
model.add(LSTM(256,input_shape=(sequence_len, 1),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences = True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
    

In [29]:
model.compile(loss='categorical_crossentropy',optimizer = 'adam', metrics = ['acc'])

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 512)          1574912   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)              

In [31]:
#creating callback to get the best model

filepath = "model-{epoch:02d}-{loss:.4f}.h5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callback = [checkpoint]

In [1]:
#training the model
model.fit(X, y, epochs = 100, batch_size = 64, callbacks = callback)

In [32]:
from tensorflow.keras import models

## Predicting and Saving the Output From The Model

In [33]:
model = models.load_model('models/model.h5')

In [35]:
start = np.random.randint(0, len(X)-1) #select a random positon to start

In [37]:
start

33426

In [40]:
#Loopup table to decode the output

int_to_note = dict()

for note, id in enumerate(pitches):
    int_to_note[note] = id


In [41]:
int_to_note

{0: '0',
 1: '0.1',
 2: '0.1.5',
 3: '0.1.6',
 4: '0.2',
 5: '0.2.3.7',
 6: '0.2.4.7',
 7: '0.2.5',
 8: '0.2.6',
 9: '0.2.7',
 10: '0.3',
 11: '0.3.5',
 12: '0.3.5.8',
 13: '0.3.6',
 14: '0.3.6.8',
 15: '0.3.6.9',
 16: '0.3.7',
 17: '0.4',
 18: '0.4.5',
 19: '0.4.6',
 20: '0.4.7',
 21: '0.5',
 22: '0.5.6',
 23: '0.6',
 24: '1',
 25: '1.2',
 26: '1.2.4.6.8.10',
 27: '1.2.6',
 28: '1.2.6.8',
 29: '1.3',
 30: '1.3.5',
 31: '1.3.5.8',
 32: '1.3.6',
 33: '1.3.7',
 34: '1.3.8',
 35: '1.4',
 36: '1.4.6',
 37: '1.4.6.9',
 38: '1.4.7',
 39: '1.4.7.10',
 40: '1.4.7.9',
 41: '1.4.8',
 42: '1.5',
 43: '1.5.8',
 44: '1.5.9',
 45: '1.6',
 46: '1.7',
 47: '10',
 48: '10.0',
 49: '10.0.2.5',
 50: '10.0.3',
 51: '10.0.4',
 52: '10.0.5',
 53: '10.1',
 54: '10.1.3',
 55: '10.1.3.5.6',
 56: '10.1.3.6',
 57: '10.1.4',
 58: '10.1.4.6',
 59: '10.1.5',
 60: '10.11',
 61: '10.11.3',
 62: '10.11.3.5',
 63: '10.2',
 64: '10.2.3',
 65: '10.2.4',
 66: '10.2.5',
 67: '10.3',
 68: '11',
 69: '11.0',
 70: '11.0.4',
 

In [43]:
X[start]

array([[0.76536313],
       [0.88268156],
       [0.98603352],
       [0.74022346],
       [0.91899441],
       [0.86592179],
       [0.23743017],
       [0.82402235],
       [0.82681564],
       [0.82681564],
       [0.46089385],
       [0.82402235],
       [0.82681564],
       [0.82681564],
       [0.58659218],
       [0.82402235],
       [0.82681564],
       [0.82681564],
       [0.20949721],
       [0.82402235],
       [0.82681564],
       [0.82681564],
       [0.23743017],
       [0.82402235],
       [0.82681564],
       [0.82402235],
       [0.82681564],
       [0.23743017],
       [0.82402235],
       [0.82681564],
       [0.82681564],
       [0.46089385],
       [0.82402235],
       [0.82681564],
       [0.82681564],
       [0.58659218],
       [0.82402235],
       [0.82681564],
       [0.82681564],
       [0.70670391],
       [0.82402235],
       [0.82681564],
       [0.82681564],
       [0.5698324 ],
       [0.82402235],
       [0.93575419],
       [0.20949721],
       [0.918

In [104]:
stream = X[start]#Selecting a random note from the input
 
predictions=[]
for i in range(100):

    stream = stream.reshape(1,100,1)#Reshaping according to model (1, sequence_length, 1)

    model_predict  = model.predict(stream)[0]
    y_pred= np.argmax(model_predict, axis=0) #Selecting the note with max probabilty
    predictions.append(y_pred)

    stream = np.insert(stream[0],len(stream[0]),y_pred) #Adding the output note to the input stream
    stream = stream[1:] #deleting the first note in the input stream
    

In [92]:
#convert the output to notes from lookup dictionary

predicted_notes = [int_to_note[i] for i in predictions]

In [93]:
len(predicted_notes)

100

In [98]:
from music21 import instrument, note

offset = 0 #used to avoid stacking of notes
output_notes = []


for stream in predicted_notes:
    # Check for note or a chord
    if ('.' in stream) or stream.isdigit():
        notes_in_chord = stream.split('.')
        notes = []
        for current_note in notes_in_chord:
            c_note = int(current_note)
            new_note = note.Note(c_note)
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
                
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
            
        
    else:
        new_note = note.Note(stream)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    offset += 0.5 # The data we used for training had a offset of 0.5

In [101]:
len(output_notes)

100

In [103]:
# Save the output as MIDI file
from music21 import stream

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='sample_outputs/example_output.mid')

'example_output.mid'